# Neighbourhood segmentation and clustering

## Import Libraries

In [115]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### Foursquere Credentials

In [120]:
CLIENT_ID = 'YPCC45XDJWEFJL5DD5BRWI0XGZCG22QVV4KJFXQZ54QWONRH' # your Foursquare ID
CLIENT_SECRET = 'X1AOOW5ZS14PNG21WFZLNUJ0TMBJBQB450U00FYOLHTCH1NF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=80

### Read Data

In [121]:
df=pd.read_csv("https://raw.githubusercontent.com/isdela/Coursera_Capstone/master/Toronto_Loc.csv")

df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.770817,-79.413300
1,M4A,North York,Victoria Village,43.770817,-79.413300
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654027,-79.380200
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.770817,-79.413300
4,M7A,Queen's Park,Scarborough,43.659980,-79.390369


### Get venues in Neighbourhood

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:
Toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
    
Toronto_venues.head()

Parkwoods
https://api.foursquare.com/v2/venues/explore?&client_id=YPCC45XDJWEFJL5DD5BRWI0XGZCG22QVV4KJFXQZ54QWONRH&client_secret=X1AOOW5ZS14PNG21WFZLNUJ0TMBJBQB450U00FYOLHTCH1NF&v=20180605&ll=43.7708175,-79.4132998&radius=500&limit=80


KeyError: 'groups'

# "errorType":"quota_exceeded","errorDetail":"Quota exceeded"

### Neighborhood by categories

In [140]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Court,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Neighborhood,New American Restaurant,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Group by Neighborhood

In [141]:

Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Court,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,New American Restaurant,Office,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.0,0.03,0.000000,0.0,0.0,0.0,0.01,0.0,0.03,0.01,0.010000,0.000000,0.02,0.0,0.020000,0.02,0.0,0.02,0.02,0.01,0.000000,0.020000,0.01,0.0,0.0,0.0,0.02,0.0,0.060000,0.0,0.060000,0.0,0.0,0.01,0.0,0.01,0.0,0.02,0.0,0.0,0.0,0.0,0.010000,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.020000,0.01,0.0,0.0,0.010000,0.0,0.000000,0.02,0.0,0.0,0.000000,0.02,0.01,0.0,0.0,0.0,0.030000,0.01,0.000000,0.0,0.030000,0.01,0.0,0.0,0.0,0.01,0.000000,0.01,0.000000,0.0,0.01,0.03,0.01,0.01,0.01,0.01,0.0,0.01,0.01,0.0,0.0,0.0,0.000000,0.010000,0.0,0.02,0.01,0.0,0.0,0.000000,0.01,0.0,0.030000,0.0,0.0,0.000000,0.020000,0.01,0.0,0.01,0.01,0.02,0.0,0.01,0.000000,0.01,0.0,0.03,0.01,0.020000,0.01,0.000000,0.0,0.0,0.01,0.000000,0.0,0.000000,0.0,0.0
1,Agincourt,0.0,0.00,0.043478,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.021739,0.086957,0.00,0.0,0.043478,0.00,0.0,0.00,0.00,0.00,0.021739,0.043478,0.00,0.0,0.0,0.0,0.00,0.0,0.065217,0.0,0.043478,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.021739,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.065217,0.00,0.0,0.0,0.021739,0.0,0.021739,0.00,0.0,0.0,0.021739,0.00,0.00,0.0,0.0,0.0,0.065217,0.00,0.021739,0.0,0.021739,0.00,0.0,0.0,0.0,0.00,0.021739,0.00,0.021739,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.021739,0.065217,0.0,0.00,0.00,0.0,0.0,0.043478,0.00,0.0,0.021739,0.0,0.0,0.021739,0.043478,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.021739,0.00,0.0,0.00,0.00,0.021739,0.00,0.021739,0.0,0.0,0.00,0.021739,0.0,0.021739,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.00,0.043478,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.021739,0.086957,0.00,0.0,0.043478,0.00,0.0,0.00,0.00,0.00,0.021739,0.043478,0.00,0.0,0.0,0.0,0.00,0.0,0.065217,0.0,0.043478,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.021739,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.065217,0.00,0.0,0.0,0.021739,0.0,0.021739,0.00,0.0,0.0,0.021739,0.00,0.00,0.0,0.0,0.0,0.065217,0.00,0.021739,0.0,0.021739,0.00,0.0,0.0,0.0,0.00,0.021739,0.00,0.021739,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.021739,0.065217,0.0,0.00,0.00,0.0,0.0,0.043478,0.00,0.0,0.021739,0.0,0.0,0.021739,0.0

### Find 10 first common values for each neighborhood

In [142]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [143]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
1,Agincourt,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
4,"Alderwood, Long Branch",Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster Neighborhood

In [144]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 2, 1, 1, 0, 0, 0, 4, 2], dtype=int32)

In [145]:
# add clustering labels
 neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [146]:
Toronto_merged = df#Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.770817,-79.413300,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
1,M4A,North York,Victoria Village,43.770817,-79.413300,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654027,-79.380200,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.770817,-79.413300,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
4,M7A,Queen's Park,Scarborough,43.659980,-79.390369,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Park,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Pharmacy


In [147]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude= location.longitude

In [148]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusterd

### Cluster 1

In [149]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
1,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
3,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
7,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
10,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
13,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
27,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
28,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
33,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant
34,North York,0,Coffee Shop,Ramen Restaurant,Bubble Tea Shop,Sushi Restaurant,Park,Pizza Place,Korean Restaurant,Sandwich Place,Café,Japanese Restaurant


### Cluster 2

In [150]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
11,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
17,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
70,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
77,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
88,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
89,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
93,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
94,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega
98,Etobicoke,1,Playground,Coffee Shop,Pharmacy,Supermarket,Garden,Dance Studio,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster 3

In [151]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
12,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
18,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
22,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
26,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
32,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
38,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
44,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
51,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade
58,Scarborough,2,Beach,Hotel,Pizza Place,Clothing Store,Fast Food Restaurant,Pub,Café,Bookstore,Seafood Restaurant,Arcade


### Cluster 4

In [152]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Queen's Park,3,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Park,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Pharmacy
8,East York,3,Coffee Shop,Italian Restaurant,Pizza Place,Café,Park,Sandwich Place,Athletics & Sports,Farmers Market,Deli / Bodega,Dessert Shop
14,East York,3,Coffee Shop,Italian Restaurant,Pizza Place,Café,Park,Sandwich Place,Athletics & Sports,Farmers Market,Deli / Bodega,Dessert Shop
23,East York,3,Coffee Shop,Italian Restaurant,Pizza Place,Café,Park,Sandwich Place,Athletics & Sports,Farmers Market,Deli / Bodega,Dessert Shop
29,East York,3,Coffee Shop,Italian Restaurant,Pizza Place,Café,Park,Sandwich Place,Athletics & Sports,Farmers Market,Deli / Bodega,Dessert Shop
35,East York,3,Coffee Shop,Italian Restaurant,Pizza Place,Café,Park,Sandwich Place,Athletics & Sports,Farmers Market,Deli / Bodega,Dessert Shop


### Cluster 5

In [153]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
9,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
15,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
16,York,4,Pub,Café,Bar,Italian Restaurant,Cocktail Bar,Historic Site,French Restaurant,Coffee Shop,Plaza,Ice Cream Shop
19,East Toronto,4,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Art Gallery,Café,Breakfast Spot,Bubble Tea Shop,Chinese Restaurant,Bar,Hotel
20,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
21,York,4,Pub,Café,Bar,Italian Restaurant,Cocktail Bar,Historic Site,French Restaurant,Coffee Shop,Plaza,Ice Cream Shop
24,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
25,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
30,Downtown Toronto,4,Clothing Store,Coffee Shop,Bakery,Tea Room,Middle Eastern Restaurant,Italian Restaurant,Hotel,Restaurant,American Restaurant,Beer Bar
